<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/numpy_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
#
#  Copyright 2025 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Numpy integration

This notebook shows how to speed up the simulation by avoiding memory copies when the simulated data must be accessed in Python.
The table below shows runtimes for the different function calls on my current system.

| Python integration | Array type | Number of images simulated in one second |
|--------------------|------------|------------------------------------------|
| False | Python | 331 |
| False | Numpy | 125 |
| True | Numpy | 1120 |



<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the package with `!pip install gvxr`
</div>

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !apt-get install libnvidia-gl-575
    !pip install gvxr

# Aims of this session

1. Revisit the example from last time;
2. Estimate the speed up factor when the Numpy integration is used.

![](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/02-visualisation.png?raw=1)

# Cheat Sheet

## Simulation parameters

| Parameters | Values | Units | Function call |
|------------|--------|-------|---------------|
| Source position | [-40, 0, 0] | cm | `gvxr.setSourcePosition(-40.0, 0.0, 0.0, "cm")` |
| Source type (beam shape) | Point source (cone beam) | | `gvxr.usePointSource()` |
| Beam spectrum | Monochromatic: 1,000 photons of 80 | keV | `gvxr.setMonoChromatic(80.0, "keV", 1000)` or |
| | | MeV | `gvxr.setMonoChromatic(0.08, "MeV", 1000)` |
| Detector position | [10, 0, 0] | cm | `gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm")` |
| Detector orientation | [0, 0, -1] |  | `gvxr.setDetectorUpVector(0, 0, -1)` |
| Detector resolution | 640 &times; 320 | pixels | `gvxr.setDetectorNumberOfPixels(640, 320)` |
| Pixel spacing | 0.5 &times; 0.5 | mm | `gvxr.setDetectorPixelSize(0.5, 0.5, "mm")` |
| Sample | STL file of the Welsh dragon | mm | `gvxr.loadMeshFile(string_ID, fname, "mm")` |
| Material composition | Ti90Al6V4 alloy |  | `gvxr.setMixture("Dragon", "Ti90Al6V4")` or |
| | | | `gvxr.setMixture("Dragon", [22, 13, 23], [0.9, 0.06, 0.04])` |
| Material density | 4.43 | g/cm3 | `gvxr.setDensity("Dragon", 4.43, "g/cm3")` or |
| | | g.cm-3 | `gvxr.setDensity("Dragon", 4.43, "g.cm-3")` |

## Other gVXR's function used

| Functions | Descriptions |
|-----------|--------------|
| `gvxr.createOpenGLContext` | Create a simulation environment automatically. You must call either `gvxr.createOpenGLContext()` or `gvxr.createNewContext()` before any other functions from gVXR. |
| `gvxr.moveToCentre` | Move a polygon mesh to the centre. |
| `gvxr.setElement` | Set the chemical element (e.g. iron) corresponding to the material properties of a polygon mesh. |
| `gvxr.setCompound` | Set the compound (e.g. water) corresponding to the material properties of a polygon mesh. |
| `gvxr.computeXRayImage` | Compute the X-ray projection corresponding to the environment that has previously been set. |
| `gvxr.displayScene` | Update the visualisation window. It works with both the interactive window and offscreen rendering. |
| `gvxr.saveLastXRayImage` | Save the last computed X-ray image in a file (e.g. TIFF, MHA, MHD/RAW) and store the data using single-precision floating-point numbers. |
| `gvxr.saveLastLBuffer` | Save the last computed path length in a file (e.g. TIFF, MHA, MHD/RAW) and store the data using single-precision floating-point numbers. The red channel is the path length; the green and blue channels can be used to detect errors (0.0 values mean no error). |
| `gvxr.setColour` | Change the colour of an object in the 3D visualisation. |
| `gvxr.setWindowBackGroundColour` | Change the background colour of the 3D visualisation. |
| `gvxr.takeScreenshot` | Take the screenshot of the current state of the 3D visualisation. |
| `gvxr.renderLoop` | Activate the interactive mode of the 3D visualisation. |
| `gvxr.terminate` | Close and destroy all the windows and simulation contexts that have been created. No further gVXR's function should be called after `gvxr.terminate`. |

## Import packages

- `os` to create the output directory if needed
- `time` and `datetime` to compute exectution durations
- `gvxr` to simulate X-ray images

In [3]:
# Import packages
import os # Create the output directory if necessary
import numpy as np # Who does not use Numpy?

import time
from datetime import timedelta

from gvxrPython3 import gvxr # Simulate X-ray images

## 1. Recycle the code from [the notebook on First X-ray simulation](notebooks/first_xray_simulation.ipynb)

In [5]:
# Create an OpenGL context
print("Create an OpenGL context");
gvxr.createOpenGLContext();

Create an OpenGL context


Mon Jul 21 14:32:20 2025 ---- Create window (ID: -1)
Mon Jul 21 14:32:20 2025 ---- Request an interactive OpenGL context
Mon Jul 21 14:32:20 2025 ---- Initialise GLFW
Mon Jul 21 14:32:20 2025 ---- Create an OpenGL window with a 4.3 context.
Mon Jul 21 14:32:21 2025 ---- Make the window's context current
Mon Jul 21 14:32:21 2025 ---- Initialise GLEW
Mon Jul 21 14:32:21 2025 ---- OpenGL vendor: NVIDIA Corporation
Mon Jul 21 14:32:21 2025 ---- OpenGL renderer: NVIDIA GeForce RTX 4060 Ti/PCIe/SSE2
Mon Jul 21 14:32:21 2025 ---- OpenGL version: 4.3.0 NVIDIA 570.153.02
Mon Jul 21 14:32:21 2025 ---- Use OpenGL 4.5.
Mon Jul 21 14:32:21 2025 ---- Use OpenGL Compute Shader.
Mon Jul 21 14:32:21 2025 ---- Initialise the X-ray renderer if needed and if possible


In [6]:
# Create a source
print("Set up the beam");
gvxr.setSourcePosition(-40.0,  0.0, 0.0, "cm");
gvxr.usePointSource();
#  For a parallel source, use gvxr.useParallelBeam();

Set up the beam


In [7]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.setMonoChromatic(0.08, "MeV", 1000);
# The following is equivalent: gvxr.setMonoChromatic(80, "keV", 1000);

Mon Jul 21 14:32:21 2025 (WW) setMonoChromatic is deprecated. It will be removed in a future release. Choose setMonoChromaticPerPixelAtSDD or setMonoChromaticPerCm2At1m. Note that setMonoChromatic and setMonoChromaticPerPixelAtSDD are equivalent.


In [8]:
# Set up the detector
print("Set up the detector");
gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm");
gvxr.setDetectorUpVector(0, 0, -1);
gvxr.setDetectorNumberOfPixels(640, 320);
gvxr.setDetectorPixelSize(0.5, 0.5, "mm");

Set up the detector


Mon Jul 21 14:32:21 2025 ---- Initialise the renderer


In [9]:
# Locate the sample STL file from the package directory
path = os.path.dirname(gvxr.__file__);
fname = path + "/welsh-dragon-small.stl";

# Load the sample data
if not os.path.exists(fname):
    raise IOError(fname);

print("Load the mesh data from", fname);
gvxr.loadMeshFile("Dragon", fname, "mm");

print("Move ", "Dragon", " to the centre");
gvxr.moveToCentre("Dragon");

Load the mesh data from /home/fpvidal/miniconda3/envs/DIAD2gVXR/lib/python3.11/site-packages/gvxrPython3/_gvxrPython3/welsh-dragon-small.stl
Move  Dragon  to the centre


Mon Jul 21 14:32:22 2025 ---- file_name:	/home/fpvidal/miniconda3/envs/DIAD2gVXR/lib/python3.11/site-packages/gvxrPython3/_gvxrPython3/welsh-dragon-small.stl	nb_faces:	457345	nb_vertices:	1372035	bounding_box (in cm):	(-4.47065, -74.9368, 23.5909)	(2.37482, -59.4256, 36.0343)


In [10]:
# Material properties
print("Set ", "Dragon", "'s material");

# Titanium Aluminum Vanadium Alloy
gvxr.setMixture("Dragon", ["Ti", "Al", "V"], [0.9, 0.06, 0.04]);
gvxr.setDensity("Dragon", 4.43, "g.cm-3");

Set  Dragon 's material


## 2. Compute the corresponding X-ray image as a Python array without the Numpy integration

We run it 250 times to gather statistically meaningful results.

In [11]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.

number_of_iterations = 250
print("Compute an X-ray image", number_of_iterations, "times")

without_numpy_integration1_start_time = time.monotonic()
for i in range(number_of_iterations):
    x_ray_image = gvxr.computeXRayImage()
without_numpy_integration1_end_time = time.monotonic()

Compute an X-ray image 250 times


## 3. Compute the corresponding X-ray image as a Numpy array without the Numpy integration

We run it 250 times to gather statistically meaningful results.

In [12]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.

print("Compute an X-ray image", number_of_iterations, "times")

without_numpy_integration2_start_time = time.monotonic()
for i in range(number_of_iterations):
    x_ray_image = np.array(gvxr.computeXRayImage(), dtype=np.single)
without_numpy_integration2_end_time = time.monotonic()

Compute an X-ray image 250 times


## 4. Compute the corresponding X-ray image as a Numpy array with the Numpy integration

We run it 250 times to gather statistically meaningful results.

In [13]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.

print("Compute an X-ray image", number_of_iterations, "times")

with_numpy_integration_start_time = time.monotonic()
x_ray_image = np.zeros((gvxr.getDetectorNumberOfPixels()[1], gvxr.getDetectorNumberOfPixels()[0]), dtype=np.single)
for i in range(number_of_iterations):
    gvxr.computeXRayImage(x_ray_image)
with_numpy_integration_end_time = time.monotonic()

Compute an X-ray image 250 times


## 5. Compare the performances

In [14]:
print("Python array without Numpy integration:", round(number_of_iterations / timedelta(seconds=without_numpy_integration1_end_time - without_numpy_integration1_start_time).total_seconds()), "images per second")
print("Numpy array without Numpy integration:", round(number_of_iterations / timedelta(seconds=without_numpy_integration2_end_time - without_numpy_integration2_start_time).total_seconds()), "images per second")
print("Numpy array with Numpy integration:", round(number_of_iterations / timedelta(seconds=with_numpy_integration_end_time - with_numpy_integration_start_time).total_seconds()), "images per second")

Python array without Numpy integration: 331 images per second
Numpy array without Numpy integration: 125 images per second
Numpy array with Numpy integration: 1120 images per second


# Cleaning up

Once we have finished, it is good practice to clean up the OpenGL contexts and windows with the following command. Note that due to the object-oriented programming nature of the core API of gVXR, this step is automatic anyway.

In [15]:
gvxr.destroy();

Mon Jul 21 14:32:25 2025 ---- Destroy all the windows
Mon Jul 21 14:32:25 2025 ---- Destroy window 0(0x563919e8a510)
Mon Jul 21 14:32:25 2025 ---- Release all the memory
